In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding
from keras import Sequential, Input
from keras.layers import LSTM, Dense, Dropout, SpatialDropout1D
import pickle
from keras.models import to_json
import os

In [2]:
train_data = pd.read_csv('dataset/train_preprocessed.csv', encoding='utf-8')
train_data.dropna()

,text,class
0,unless request information withheld comply fer...,5
1,none collect personal information computer e m...,5
2,elect location based search saved history stor...,6
3,subsidiary corporate affiliate including enfor...,5
4,use service view content provided google autom...,2
...,...,...
13545,opt targeted advertising,6
13546,web page computer visit using service clickstr...,5
13547,jibjab message sent visiting adjusting email p...,6
13548,receive store certain type information wheneve...,2


In [3]:
Y = pd.get_dummies(train_data['class']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (13550, 10)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(train_data['text'], Y, 
                                                    test_size=0.3, random_state=42)

In [5]:
vectorizer = TextVectorization(max_tokens=2500, output_sequence_length=40)
text_ds = tf.data.Dataset.from_tensor_slices(X_train).batch(128)
vectorizer.adapt(text_ds)

In [6]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [7]:
embeddings_index = {}
with open('GloVe/glove.6B.50d.txt', encoding="utf-8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 400000 word vectors.


In [8]:
num_tokens = len(voc) + 2
embedding_dim = 50
hits = 0
misses = 0

embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 2426 words (74 misses)


In [9]:
embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

In [10]:
X_train_transformed = vectorizer(np.array([[s] for s in X_train])).numpy()
X_test_transformed = vectorizer(np.array([[s] for s in X_test])).numpy()

y_train_transformed = np.array(y_train)
y_test_transformed = np.array(y_test)

In [11]:
print(X_train_transformed.shape)
print(X_test_transformed.shape)
print(y_train_transformed.shape)
print(y_test_transformed.shape)

(9485, 40)
(4065, 40)
(9485, 10)
(4065, 10)


In [12]:
model = Sequential()
model.add(embedding_layer)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          125100    
                                                                 
 spatial_dropout1d (SpatialD  (None, None, 50)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               60400     
                                                                 
 dense (Dense)               (None, 32)                3232      
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 189,062
Trainable params: 63,962
Non-trainable params: 125,100
_____________________________________________

In [13]:
model.fit(X_train_transformed, y_train_transformed, batch_size=64, epochs=50, 
          validation_data=(X_test_transformed, y_test_transformed))

Epoch 1/50
149/149 [==============================] - 11s 47ms/step - loss: 1.7371 - accuracy: 0.3986 - val_loss: 1.6241 - val_accuracy: 0.4273
Epoch 2/50
149/149 [==============================] - 6s 41ms/step - loss: 1.5649 - accuracy: 0.4781 - val_loss: 1.4661 - val_accuracy: 0.5319
Epoch 3/50
149/149 [==============================] - 6s 41ms/step - loss: 1.4529 - accuracy: 0.5398 - val_loss: 1.3283 - val_accuracy: 0.5833
Epoch 4/50
149/149 [==============================] - 6s 40ms/step - loss: 1.3510 - accuracy: 0.5769 - val_loss: 1.2507 - val_accuracy: 0.5975
Epoch 5/50
149/149 [==============================] - 6s 40ms/step - loss: 1.2538 - accuracy: 0.5994 - val_loss: 1.1971 - val_accuracy: 0.6133
Epoch 6/50
149/149 [==============================] - 6s 41ms/step - loss: 1.1910 - accuracy: 0.6137 - val_loss: 1.0852 - val_accuracy: 0.6433
Epoch 7/50
149/149 [==============================] - 6s 41ms/step - loss: 1.1755 - accuracy: 0.6134 - val_loss: 1.1101 - val_accuracy: 0.636

In [16]:
model.save("models/LSTM_GloVe_keras.h5")

In [18]:
pickle.dump({'config': vectorizer.get_config(),
             'weights': vectorizer.get_weights()}
            , open("models/LSTM_GloVe_Vectorizer.pkl", "wb"))